Model 600 BasicSaVs

Charlie Shobe

Though we have already tested these two components individually, this model needs to be tested because the VSA formulation acts differently as the soil layer changes its thickness.

There will not be an analytical solution here, but the final result will be a plot showing:

1) The detachment-limited slope area solution:

$$S=\left(\frac{U}{K}\right)^{1/n}A^{-m/n}$$

2) The slope-area data from the steady-state model domain

3) The relationship between drainage area (unaltered) and slope (predicted by the VSA equation given the soil thickness)

In [1]:
from terrainbento import BasicSaVs
import numpy as np
import matplotlib.pyplot as plt
from landlab import imshow_grid

//anaconda/lib/python2.7/site-packages/xarray/core/formatting.py:16: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


In [2]:
#parameter dictionary 
params = { 'number_of_node_rows' : 100,
          'number_of_node_columns' : 160,
          'node_spacing' : 10.0,
          'east_boundary_closed' : False,
          'north_boundary_closed' : False,
          'west_boundary_closed' : False,
          'south_boundary_closed' : False,
          'dt' : 10.0,
          'K_sp' : 0.001,
          'm_sp' : 0.5,
          'n_sp' : 1.0,
          'linear_diffusivity' : 0.01,
          'outlet_lowering_rate' : 0.0005,
          'output_filename': 'model_000_output',
          'recharge_rate': 0.01,
          'initial_soil_thickness' : 1.0,
          'K_hydraulic_conductivity' : 15.,
          'soil_transport_decay_depth' : 0.1,
          'max_soil_production_rate' : 0.01,
          'soil_production_decay_depth' : 0.5, 
          'initial_soil_thickness' : 0.0
}

In [3]:
#instantiate and establish topo error tolerance
vssa = BasicSaVs(params=params)
tolerance = 0.0001

In [4]:
#time loop
dt = 10.
elapsed_time = 0 #years
keep_running = True
while keep_running == True:
    pre_topo = vssa.grid.at_node['topographic__elevation'][vssa.grid.core_nodes]
    vssa.run_one_step(dt)
    post_topo = vssa.grid.at_node['topographic__elevation'][vssa.grid.core_nodes]
    if elapsed_time % 1000 == 0:
        print(elapsed_time)
        print('MAX: ', max(abs(pre_topo - post_topo)))
        print('MEAN: ', np.average(abs(pre_topo - post_topo)))
    elapsed_time += dt
    if max(abs(pre_topo - post_topo)) <= tolerance: #1mm
        keep_running = False

0
MAX:  0.112142200375
MEAN:  0.00301307196371
1000.0
MAX:  0.0114677985562
MEAN:  0.00450376775746
2000.0
MAX:  0.0165308228836
MEAN:  0.00432578070238
3000.0
MAX:  0.00581851228044
MEAN:  0.00414307390906
4000.0
MAX:  0.0063678722017
MEAN:  0.00398442027758
5000.0
MAX:  0.0197345012069
MEAN:  0.00385096162641
6000.0
MAX:  0.00683174606718
MEAN:  0.00373398025956
7000.0
MAX:  0.00782786596941
MEAN:  0.00363468990565
8000.0
MAX:  0.00796733008983
MEAN:  0.00354537431293
9000.0
MAX:  0.00822238582301
MEAN:  0.00346650912597
10000.0
MAX:  0.00736088670911
MEAN:  0.00339682722427
11000.0
MAX:  0.00837135453499
MEAN:  0.00333126405146
12000.0
MAX:  0.00840458343927
MEAN:  0.00327058658328
13000.0
MAX:  0.00958073042424
MEAN:  0.00321430425648
14000.0
MAX:  0.0110850449695
MEAN:  0.00315937318123
15000.0
MAX:  0.0111818714675
MEAN:  0.00310881822238
16000.0
MAX:  0.0103537656337
MEAN:  0.00306068920921
17000.0
MAX:  0.0119807276259
MEAN:  0.00301739054767
18000.0
MAX:  0.0109447726541
MEAN:

KeyboardInterrupt: 

In [ ]:
%matplotlib inline
imshow_grid(vssa.grid, 'topographic__elevation')

In [ ]:
print(vssa.grid.at_node['topographic__steepest_slope'][235])
#for some reason, the line of nodes adjacent to all the boundaries
plt.scatter(vssa.grid.at_node['drainage_area'][(vssa.grid.node_x > 20)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >20)&(vssa.grid.node_y<980)], vssa.grid.at_node['topographic__steepest_slope'][(vssa.grid.node_x > 20)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >20)&(vssa.grid.node_y<980)])
plt.plot(vssa.grid.at_node['drainage_area'][(vssa.grid.node_x > 20)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >20)&(vssa.grid.node_y<980)], (0.0005 / 0.005)*np.power(vssa.grid.at_node['drainage_area'][(vssa.grid.node_x > 20)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >20)&(vssa.grid.node_y<980)], -0.5), c='r')
plt.xscale('log')
plt.yscale('log')
plt.ylim(0.0001, 1)

In [ ]:
imshow_grid(vssa.grid, 'soil__depth')
print(vssa.grid.at_node['soil__depth'][vssa.grid.core_nodes])

In [ ]:

#first: the data
plt.scatter(vssa.grid.at_node['drainage_area'][(vssa.grid.node_x > 20)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >20)&(vssa.grid.node_y<980)], vssa.grid.at_node['topographic__steepest_slope'][(vssa.grid.node_x > 20)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >20)&(vssa.grid.node_y<980)])

#second: the detachment-limited slope-area relationship
plt.plot(vssa.grid.at_node['drainage_area'][vssa.grid.core_nodes], (0.0005 / 0.005) *np.power(vssa.grid.at_node['drainage_area'][vssa.grid.core_nodes],-0.5), c='r')

#third: calculate the slope predicted by Aeff
slope_by_aeff = (0.0005 / 0.005) * np.power(vssa.eff_area, -0.5)
#print(len(slope_by_aeff[vs.grid.at_node['drainage_area'][vs.grid.core_nodes]<=1000]))
#print(len((0.0005 / 0.005) *np.power(vs.grid.at_node['drainage_area'][vs.grid.core_nodes],-0.5)[vs.grid.at_node['drainage_area'][vs.grid.core_nodes]<=1000]))

#print(slope_by_aeff[vs.grid.at_node['drainage_area'][vs.grid.core_nodes]<=1000]# /( (0.0005 / 0.005) *np.power(vs.grid.at_node['drainage_area'][vs.grid.core_nodes],-0.5)[vs.grid.at_node['drainage_area'][vs.grid.core_nodes]<=1000]))


#plt.scatter(vs.eff_area[vs.grid.core_nodes], slope_by_aeff)

plt.scatter(vssa.grid.at_node['drainage_area'][(vssa.grid.node_x > 20)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >20)&(vssa.grid.node_y<980)], slope_by_aeff[(vssa.grid.node_x > 20)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >20)&(vssa.grid.node_y<980)])

plt.xscale('log')
plt.yscale('log')
plt.xlim(90, 100000)
plt.ylim(0.0001, 0.1)

In [5]:
###MAKE SLOPE-AREA PLOT

import matplotlib

#assign area_array and slope_array
area_array = vssa.grid.at_node['drainage_area'][(vssa.grid.node_x > 10)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >10)&(vssa.grid.node_y<980)]
slope_array = vssa.grid.at_node['topographic__steepest_slope'][(vssa.grid.node_x > 10)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >10)&(vssa.grid.node_y<980)]

#plotting param
matplotlib.rcParams.update({'font.size': 20})

##instantiate figure and plot
fig = plt.figure(figsize=(6, 3.75))
slope_area = plt.subplot()

#create an array for the detachment-limited analytical solution
u = 0.0005 #m/yr, uplift or baselevel lowering rate
k = 0.001 #fluvial erodibility
m = 0.5 #discharge exponent
n = 1.0 #slope exponent

#calculate analytical slope from area field
analytical_slope_array = np.power((u / k), 1 / n) * np.power(area_array, -m/n)

#plot the slope predicted by the effective area
slope_by_aeff = (0.0005 / 0.001) * np.power(vssa.eff_area, -0.5)
plt.scatter(vssa.grid.at_node['drainage_area'][(vssa.grid.node_x > 10)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >10)&(vssa.grid.node_y<980)], 
            slope_by_aeff[(vssa.grid.node_x > 10)&(vssa.grid.node_x < 1580)&(vssa.grid.node_y >10)&(vssa.grid.node_y<980)],
           label = 'VSA solution',
           marker='o', edgecolor='0', color='1')

#plot the data
slope_area.scatter(area_array, slope_array, marker='o', c='k', 
                   label = 'VSA + diffusion solution') #plot HA data


#plot the analytical solution
slope_area.plot(area_array, analytical_slope_array, linestyle='-',
                color='grey', linewidth = 1, label = 'Basic solution')

#make axes log and set limits
slope_area.set_xscale('log')
slope_area.set_yscale('log')

slope_area.set_xlim(9*10**1, 1*10**6)
slope_area.set_ylim(1e-4, 1e4)

#set x and y labels
slope_area.set_xlabel(r'Drainage area [m$^2$]')
slope_area.set_ylabel('Channel slope [-]')
slope_area.legend(scatterpoints=1,prop={'size':12})
slope_area.tick_params(axis='x', which='major', pad=7)

fig.savefig('vsa_soil_alluvium_slope_area.eps',bbox_inches='tight', dpi=1000) #save figure

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: RuntimeWarning: divide by zero encountered in power


In [6]:
##MAKE TOPO FIGURE
#define 'topo'
topo = vssa.grid.at_node['topographic__elevation']
#plotting param
matplotlib.rcParams.update({'font.size': 20})

# #instantiate figure and plot
topo_fig = plt.figure(figsize=(6, 3.75))
t1 = plt.subplot()
topo = topo.reshape(100,160)
ts1_plot = t1.imshow(topo[::-1], cmap='terrain', vmin = 0, vmax = 270.0)

#add colorbar
cb = plt.colorbar(ts1_plot, label = 'Elevation [m]')

#axis labels
t1.tick_params(labelbottom='off', labelleft='off') 
t1.set_ylabel('1 km side length', labelpad = 15)
t1.set_xlabel('1.6 km side length', labelpad = 15)

#save figure
topo_fig.savefig('vsa_soil_alluvium_topo.eps',bbox_inches='tight', dpi=300)